# CIFAR Experiments

Notebook for evaluating our models on the classic CIFAR dataset.

## Setup

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset
import torchvision.transforms as T

import time
import numpy as np
import matplotlib.pyplot as plt

from data_utils import get_CIFAR10_data

In [14]:
%load_ext autoreload
%autoreload 2

from modulesStackable import OctConv2dStackable, get_stacked_4
from modulesStackable import get_SixLayerConvNet, get_SixLayerOctConvNet
from octconv_tests import test_octconv_shapes, test_octconv_as_conv


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
#231N Assignment 2

USE_GPU = True
print_every = 100

dtype = torch.float32

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


## Load CIFAR Data

The following code should download the CIFAR data automatically. If not, you can do so by navigating to the `datasets` directory and running the `$ ./get_datasets.sh` command from the command line.

In [4]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

0it [00:00, ?it/s]

100%|█████████▉| 169967616/170498071 [00:46<00:00, 5262001.88it/s]

Files already downloaded and verified
Files already downloaded and verified


170500096it [01:00, 5262001.88it/s]                               

In [12]:
cifar10_train[0][0].shape

torch.Size([3, 32, 32])

## Train Vanilla Model on CIFAR

We're training a pretty vanilla six-layer convolutional network on CIFAR. Implementation details can be found in modulesStackable.py.

In [5]:
def check_accuracy(loader, model):
    """
    Checks accuracy of model on given data loader.
    Code from 231N Assignment 2.
    """
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [6]:
def train_cifar(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    Code from 231N assignment 2.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy(loader_val, model)
                print()

In [7]:
vanilla_model = get_SixLayerConvNet()

learning_rate = 5e-4
optimizer = optim.Adam(vanilla_model.parameters(), lr=learning_rate)

train_cifar(vanilla_model, optimizer, epochs=10)

Iteration 0, loss = 3.5627
Checking accuracy on validation set
Got 82 / 1000 correct (8.20)

Iteration 100, loss = 1.8861
Checking accuracy on validation set
Got 281 / 1000 correct (28.10)

Iteration 200, loss = 1.7070
Checking accuracy on validation set
Got 401 / 1000 correct (40.10)

Iteration 300, loss = 1.6364
Checking accuracy on validation set
Got 441 / 1000 correct (44.10)

Iteration 400, loss = 1.6115
Checking accuracy on validation set
Got 449 / 1000 correct (44.90)

Iteration 500, loss = 1.8239
Checking accuracy on validation set
Got 473 / 1000 correct (47.30)

Iteration 600, loss = 1.3950
Checking accuracy on validation set
Got 493 / 1000 correct (49.30)

Iteration 700, loss = 1.2438
Checking accuracy on validation set
Got 494 / 1000 correct (49.40)

Iteration 0, loss = 1.2272
Checking accuracy on validation set
Got 509 / 1000 correct (50.90)

Iteration 100, loss = 1.2776
Checking accuracy on validation set
Got 517 / 1000 correct (51.70)

Iteration 200, loss = 1.2791
Checkin

In [8]:
vanilla_model.parameters()

<generator object Module.parameters at 0x7efe80337750>

## Train OctConv Model on CIFAR

We train a model with the same architecture but using OctConv models instead.

In [20]:
alpha = 0.25
hidden_channels = 32
C, H, W = cifar10_train[0][0].shape
fc_1 = 32
D_out = 10
octconv_model = get_SixLayerOctConvNet(alpha, hidden_channels, C, H, W, fc_1, D_out)

learning_rate = 5e-4
optimizer = optim.Adam(octconv_model.parameters(), lr=learning_rate)

train_cifar(octconv_model, optimizer, epochs=10)

Iteration 0, loss = 2.3210
Checking accuracy on validation set
Got 87 / 1000 correct (8.70)

Iteration 100, loss = 1.8276
Checking accuracy on validation set
Got 307 / 1000 correct (30.70)

Iteration 200, loss = 1.6957
Checking accuracy on validation set
Got 384 / 1000 correct (38.40)

Iteration 300, loss = 1.5512
Checking accuracy on validation set
Got 425 / 1000 correct (42.50)

Iteration 400, loss = 1.6306
Checking accuracy on validation set
Got 454 / 1000 correct (45.40)

Iteration 500, loss = 1.3868
Checking accuracy on validation set
Got 452 / 1000 correct (45.20)

Iteration 600, loss = 1.3454
Checking accuracy on validation set
Got 522 / 1000 correct (52.20)

Iteration 700, loss = 1.3508
Checking accuracy on validation set
Got 494 / 1000 correct (49.40)

Iteration 0, loss = 1.3938
Checking accuracy on validation set
Got 524 / 1000 correct (52.40)

Iteration 100, loss = 1.1891
Checking accuracy on validation set
Got 534 / 1000 correct (53.40)

Iteration 200, loss = 1.3636
Checkin